In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.0-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   1.22 MB/s
branca-0.3.0-p 100% |################################| Time: 0:00:00  18.84 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  28.95 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  37.65 MB/s


In [3]:
import requests
import numpy as np
import pandas as pd

import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [37]:
response = requests.get('https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json').json()

In [17]:
response['features'][0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [19]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

ny_neighborhood = pd.DataFrame(columns=column_names)

In [22]:
for data in response['features']:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    ny_neighborhood = ny_neighborhood.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [32]:
ny_neighborhood.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [81]:
project.save_data(data=ny_neighborhood.to_csv(index=False), file_name='ny_neighborhood.csv', overwrite=True)

{'asset_id': '8d95a9f1-5680-466e-a128-0e49458ba06a',
 'bucket_name': 'courseracapstoneproject-donotdelete-pr-d28xfsomlxxcfy',
 'file_name': 'ny_neighborhood.csv',
 'message': 'File ny_neighborhood.csv has been written successfully to the associated OS'}

In [6]:
nlat = 40.7308619
nlng = -73.9871558

In [30]:
map_ny = folium.Map(location=[nlat, nlng], zoom_start=10)

for lat, lng, neighborhood, borough in zip(ny_neighborhood['Latitude'], ny_neighborhood['Longitude'], ny_neighborhood['Neighborhood'], ny_neighborhood['Borough']):
    label = folium.Popup(neighborhood+', '+borough, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)
    
map_ny

In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
cat = requests.get('https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION)).json()


In [10]:
print(cat['response']['categories'])

[{'name': 'Arts & Entertainment', 'icon': {'suffix': '.png', 'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_'}, 'id': '4d4b7104d754a06370d81259', 'categories': [{'name': 'Amphitheater', 'icon': {'suffix': '.png', 'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_'}, 'id': '56aa371be4b08b9a8d5734db', 'categories': [], 'pluralName': 'Amphitheaters', 'shortName': 'Amphitheater'}, {'name': 'Aquarium', 'icon': {'suffix': '.png', 'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/aquarium_'}, 'id': '4fceea171983d5d06c3e9823', 'categories': [], 'pluralName': 'Aquariums', 'shortName': 'Aquarium'}, {'name': 'Arcade', 'icon': {'suffix': '.png', 'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/arcade_'}, 'id': '4bf58dd8d48988d1e1931735', 'categories': [], 'pluralName': 'Arcades', 'shortName': 'Arcade'}, {'name': 'Art Gallery', 'icon': {'suffix': '.png', 'prefix': 'https://ss3.4sqi.net/img/categories_v2/

In [11]:
categories = []
sub_cat = {}

for c in cat['response']['categories']:
    categories.append(c['name'])
    sub_cat[c['name']] = []
    sub_cat[c['name']].append(c['name'])
    for sub in c['categories']:
        sub_cat[c['name']].append(sub['name'])
    
print(categories)
print(sub_cat)

['Arts & Entertainment', 'College & University', 'Event', 'Food', 'Nightlife Spot', 'Outdoors & Recreation', 'Professional & Other Places', 'Residence', 'Shop & Service', 'Travel & Transport']
{'Professional & Other Places': ['Professional & Other Places', 'Animal Shelter', 'Art Studio', 'Auditorium', 'Ballroom', 'Building', 'Business Center', 'Club House', 'Community Center', 'Convention Center', 'Cultural Center', 'Distillery', 'Distribution Center', 'Event Space', 'Factory', 'Fair', 'Funeral Home', 'Government Building', 'Industrial Estate', 'Laboratory', 'Library', 'Medical Center', 'Military Base', 'Non-Profit', 'Observatory', 'Office', 'Parking', 'Post Office', 'Power Plant', 'Prison', 'Radio Station', 'Recruiting Agency', 'Research Station', 'School', 'Social Club', 'Spiritual Center', 'TV Station', 'Voting Booth', 'Warehouse', 'Waste Facility', 'Wedding Hall', 'Winery'], 'Food': ['Food', 'Afghan Restaurant', 'African Restaurant', 'American Restaurant', 'Asian Restaurant', 'Aust

In [12]:
sub_cat[categories[0]].extend(['Yoga Studio', 'Monument / Landmark', 'Tennis Court', 'History Museum', 'Dance Studio', 'Theater', 
                               'Cycle Studio', 'Basketball Court', 'Martial Arts Dojo', 'Indie Movie Theater', 'Jazz Club', 
                               'Baseball Field'])

sub_cat[categories[3]].extend(['Peruvian Restaurant', 'Korean Restaurant', 'Cuban Restaurant', 'Cupcake Shop', 'Chinese Restaurant',
                              'Sushi Restaurant', 'New American Restaurant', 'Tapas Restaurant', 'Thai Restaurant', 'Noodle House',
                              'South American Restaurant', 'Frozen Yogurt Shop', 'Japanese Restaurant', 'Taco Place',
                               'Ramen Restaurant', 'Vietnamese Restaurant', 'Filipino Restaurant', 'Ice Cream Shop'])

sub_cat[categories[4]].extend(['Beer Bar', 'Karaoke Bar', 'Hotel Bar', 'Cocktail Bar', 'Hookah Bar', 'Wine Bar', 'Sports Bar'
                              'Beer Garden', 'Dive Bar', 'Pub'])

sub_cat[categories[8]].extend(['Kids Store', 'Fish Market', 'Shoe Store', 'Lingerie Store', 'Boutique', 'Mens Store', 'Liquor Store',
                               'Womens Store', 'Farmers Market', 'Accessories Store', 'Gourmet Shop', 'Grocery Store', 'Supermarket',
                               'Wine Shop', 'Health Food Store'])

sub_cat[categories[9]].extend(['Bus Line'])

In [174]:
limit = 200 # limit of number of venues returned by Foursquare API
radius = 500
nearby_venues = []

for lat, lng, borough, neighborhood in zip(ny_neighborhood['Latitude'], ny_neighborhood['Longitude'], ny_neighborhood['Borough'], ny_neighborhood['Neighborhood']):    
    #build url
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius,
        limit)
    
    #get json data
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    
    nearby_venues.append([(
        borough,
        neighborhood,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in venues])
    print(len(venues), end=" ")

8 17 20 5 7 71 21 31 26 5 20 26 24 41 28 12 89 20 22 35 25 21 12 7 10 27 15 10 9 5 41 31 14 27 93 8 23 34 16 20 8 11 23 25 21 6 85 27 35 100 26 44 29 18 24 23 11 33 27 80 18 34 72 25 100 100 100 49 69 80 74 22 9 13 9 18 3 10 18 47 22 5 19 9 96 5 100 88 44 27 43 8 16 52 27 70 100 100 22 67 100 89 54 57 37 43 43 100 100 100 24 100 100 100 100 100 100 100 100 64 100 100 100 100 59 45 100 95 100 100 69 82 34 39 18 38 43 21 59 67 45 15 30 40 4 42 27 30 13 43 4 72 20 46 22 21 21 21 13 11 42 26 17 17 10 20 18 14 14 13 35 4 6 30 12 41 16 15 48 5 43 9 5 5 27 14 27 12 4 25 14 23 2 1 14 23 25 25 13 32 23 36 4 1 6 7 5 0 14 25 18 25 2 20 39 15 21 7 23 4 5 51 22 19 6 3 4 4 18 24 18 28 10 30 10 9 12 15 5 31 12 6 5 11 5 4 43 100 100 100 100 5 23 17 8 2 6 0 10 16 20 5 37 28 18 11 13 18 28 35 11 100 77 100 77 17 100 100 19 63 28 12 14 87 12 8 4 4 13 41 18 8 6 10 3 17 8 15 31 30 22 60 18 4 15 6 

In [175]:
ny_venues_df = pd.DataFrame([item for nearby_venues in nearby_venues for item in nearby_venues])
ny_venues_df.columns = ['Borough', 'Neighborhood', 'NeighborhoodLatitude', 'NeighborhoodLongitude', 'Venue', 'VenueLatitude', 'VenueLongitude', 'VenueSubCategory']

In [178]:
ny_venues_df.head(5)

,Borough,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueSubCategory
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy
2,Bronx,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898283,-73.850478,Caribbean Restaurant
3,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Bronx,Wakefield,40.894705,-73.847201,Dunkin Donuts,40.890631,-73.849027,Donut Shop


In [179]:
VenueCategory = np.empty([ny_venues_df.shape[0],1], dtype=str)
ny_venues_df['VenueCategory'] = VenueCategory
ny_venues_df.head()

,Borough,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueSubCategory,VenueCategory
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop,
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy,
2,Bronx,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898283,-73.850478,Caribbean Restaurant,
3,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop,
4,Bronx,Wakefield,40.894705,-73.847201,Dunkin Donuts,40.890631,-73.849027,Donut Shop,


In [180]:
for i in range(ny_venues_df.shape[0]):
    for ele in categories:
        if ny_venues_df.iloc[i,7] in sub_cat[ele]:
            ny_venues_df.iloc[i,8] = ele
            break

In [182]:
ny_venues_df.head()

,Borough,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueSubCategory,VenueCategory
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop,Food
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy,Shop & Service
2,Bronx,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898283,-73.850478,Caribbean Restaurant,Food
3,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop,Food
4,Bronx,Wakefield,40.894705,-73.847201,Dunkin Donuts,40.890631,-73.849027,Donut Shop,Food


In [189]:
ny_venues_df['VenueCategory'].replace('', np.nan, inplace=True)
ny_venues_df.dropna(inplace=True)
ny_venues_df.reset_index(drop=True, inplace=True)

In [191]:
ny_venues_df.groupby(['Borough', 'VenueCategory']).count()

Neighborhood  NeighborhoodLatitude  \
Borough       VenueCategory                                                     
Bronx         Arts & Entertainment                   31                    31   
              Food                                  657                   657   
              Nightlife Spot                         33                    33   
              Outdoors & Recreation                  50                    50   
              Professional & Other Places             4                     4   
              Shop & Service                        332                   332   
              Travel & Transport                     67                    67   
Brooklyn      Arts & Entertainment                  142                   142   
              Food                                 1431                  1431   
              Nightlife Spot                        197                   197   
              Outdoors & Recreation                 138                   138   
              Professional & Other Places            11                    11   
              Residence                               2                     2   
              Shop & Service                        677                   677   
              Travel & Transport                     61                    61   
Manhattan     Arts & Entertainment                  234                   234   
              College & University                    4                     4   
              Food                                 1667                  1667   
              Nightlife Spot                        219                   219   
              Outdoors & Recreation                 150                   150   
              Professional & Other Places            29                    29   
              Residence                               2                     2   
              Shop & Service                        594                   594   
              Travel & Transport                     79                    79   
Queens        Arts & Entertainment                   53                    53   
              College & University                    3                     3   
              Food                                 1121                  1121   
              Nightlife Spot                        113                   113   
              Outdoors & Recreation                 104                   104   
              Professional & Other Places             5                     5   
              Residence                               2                     2   
              Shop & Service                        492                   492   
              Travel & Transport                     92                    92   
Staten Island Arts & Entertainment                   30                    30   
              Food                                  400                   400   
              Nightlife Spot                         22                    22   
              Outdoors & Recreation                  41                    41   
              Professional & Other Places             2                     2   
              Residence                               1                     1   
              Shop & Service                        206                   206   
              Travel & Transport                     88                    88   

                                           NeighborhoodLongitude  Venue  \
Borough       VenueCategory                                               
Bronx         Arts & Entertainment                            31     31   
              Food                                           657    657   
              Nightlife Spot                                  33     33   
              Outdoors & Recreation                           50     50   
              Professional & Other Places                      4      4   
              Shop & Service      

In [192]:
project.save_data(data=ny_venues_df.to_csv(index=False), file_name='newyork_venues.csv', overwrite=True)

{'asset_id': '3d116d58-1cf4-401f-8196-e2644b2193ac',
 'bucket_name': 'courseracapstoneproject-donotdelete-pr-d28xfsomlxxcfy',
 'file_name': 'newyork_venues.csv',
 'message': 'File newyork_venues.csv has been written successfully to the associated OS'}

In [4]:
ny_venues_df = pd.read_csv(project.get_file('newyork_venues.csv'))

In [5]:
ny_venues_df.head(10)

,Borough,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueSubCategory,VenueCategory
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop,Food
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy,Shop & Service
2,Bronx,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898283,-73.850478,Caribbean Restaurant,Food
3,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop,Food
4,Bronx,Wakefield,40.894705,-73.847201,Dunkin Donuts,40.890631,-73.849027,Donut Shop,Food
5,Bronx,Wakefield,40.894705,-73.847201,SUBWAY,40.890656,-73.849192,Sandwich Place,Food
6,Bronx,Wakefield,40.894705,-73.847201,Baychester Avenue Food Truck,40.892293,-73.843230,Food Truck,Food
7,Bronx,Wakefield,40.894705,-73.847201,Louis Pizza,40.898457,-73.848770,Pizza Place,Food
8,Bronx,Co-op City,40.874294,-73.829939,Dollar Tree,40.870125,-73.828989,Discount Store,Shop & Service
9,Bronx,Co-op City,40.874294,-73.829939,Capri II Pizza,40.876374,-73.829940,Pizza Place,Food


In [47]:
x = pd.get_dummies(ny_venues_df['VenueCategory'])
x['Neighborhood'] = ny_venues_df['Neighborhood']
y = x.groupby('Neighborhood').sum().reset_index()
y

,Neighborhood,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport
0,Allerton,1,0,16,0,1,0,0,11,2
1,Annadale,1,0,8,1,1,0,0,2,1
2,Arden Heights,0,0,2,0,2,0,0,1,1
3,Arlington,0,0,2,0,0,0,0,0,2
4,Arrochar,0,0,10,0,0,0,0,3,5
5,Arverne,0,0,4,0,2,0,0,1,3
6,Astoria,2,0,62,16,0,0,0,13,0
7,Astoria Heights,1,0,6,0,2,0,0,2,1
8,Auburndale,0,0,7,2,1,0,0,7,1
9,Bath Beach,0,0,29,1,1,0,0,13,1


In [82]:
clusters = 3
ny_clusters = y.drop(columns=['Neighborhood'])
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(ny_clusters)
kmeans.labels_

array([2, 0, 0, 0, 0, 0, 1, 0, 0, 2, 1, 1, 2, 0, 1, 0, 2, 2, 0, 0, 0, 0, 1,
       2, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 1, 1, 0,
       0, 2, 0, 1, 1, 0, 2, 1, 0, 0, 0, 1, 1, 0, 1, 2, 0, 2, 2, 0, 0, 0, 0,
       2, 2, 2, 0, 1, 1, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 2, 0, 0, 2, 2, 0,
       0, 2, 0, 1, 2, 1, 0, 0, 2, 1, 2, 2, 1, 1, 0, 0, 2, 2, 0, 0, 0, 2, 1,
       0, 2, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 1, 0,
       2, 1, 2, 0, 2, 2, 2, 0, 1, 2, 0, 0, 1, 0, 2, 0, 1, 2, 2, 0, 2, 0, 0,
       0, 0, 1, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 2, 0, 2, 0, 2, 2, 0, 2,
       0, 1, 0, 0, 2, 0, 0, 0, 1, 0, 2, 1, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 1,
       2, 2, 0, 2, 0, 0, 0, 0, 0, 1, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 0,
       2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 2, 0, 2, 0, 2, 1, 2, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2,
       1, 1, 0, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 1,
       1], d

In [83]:
ny_merged = y

ny_merged['Cluster'] = kmeans.labels_

ny_merged['Latitude'] = np.zeros(y.shape[0])
ny_merged['Longitude'] = np.zeros(y.shape[0])
ny_merged.head()

,Neighborhood,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport,Cluster,Latitude,Longitude
0,Allerton,1,0,16,0,1,0,0,11,2,2,0.0,0.0
1,Annadale,1,0,8,1,1,0,0,2,1,0,0.0,0.0
2,Arden Heights,0,0,2,0,2,0,0,1,1,0,0.0,0.0
3,Arlington,0,0,2,0,0,0,0,0,2,0,0.0,0.0
4,Arrochar,0,0,10,0,0,0,0,3,5,0,0.0,0.0


In [84]:
ny_df = pd.read_csv(project.get_file('ny_neighborhood.csv'))
ny_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [85]:
ny_df.sort_values(by='Neighborhood', inplace=True)

i = 0

for k in range(y.shape[0]):
    if ny_merged.iloc[k, 0] == ny_df.iloc[i,1]:
        ny_merged.iloc[k,11] = ny_df.iloc[i,2]
        ny_merged.iloc[k,12] = ny_df.iloc[i,3]
        i+=1
    else:
        i+=1
        if ny_merged.iloc[k, 0] == ny_df.iloc[i,1]:
            ny_merged.iloc[k,11] = ny_df.iloc[i,2]
            ny_merged.iloc[k,12] = ny_df.iloc[i,3]
            i+=1
            
            
ny_merged

,Neighborhood,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport,Cluster,Latitude,Longitude
0,Allerton,1,0,16,0,1,0,0,11,2,2,40.865788,-73.859319
1,Annadale,1,0,8,1,1,0,0,2,1,0,40.538114,-74.178549
2,Arden Heights,0,0,2,0,2,0,0,1,1,0,40.549286,-74.185887
3,Arlington,0,0,2,0,0,0,0,0,2,0,40.635325,-74.165104
4,Arrochar,0,0,10,0,0,0,0,3,5,0,40.596313,-74.067124
5,Arverne,0,0,4,0,2,0,0,1,3,0,40.589144,-73.791992
6,Astoria,2,0,62,16,0,0,0,13,0,1,40.768509,-73.915654
7,Astoria Heights,1,0,6,0,2,0,0,2,1,0,40.770317,-73.894680
8,Auburndale,0,0,7,2,1,0,0,7,1,0,40.761730,-73.791762
9,Bath Beach,0,0,29,1,1,0,0,13,1,2,40.599519,-73.998752


In [87]:
map_cluster = folium.Map(location=[nlat, nlng], zoom_start=10)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i+x+(i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster']):
    label = folium.Popup(str(cluster)+' '+poi, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster)
       
map_cluster

In [88]:
cluster1 = ny_merged.loc[ny_merged['Cluster'] == 0, ny_merged.columns[:10]].reset_index()
cluster2 = ny_merged.loc[ny_merged['Cluster'] == 1, ny_merged.columns[:10]].reset_index()
cluster3 = ny_merged.loc[ny_merged['Cluster'] == 2, ny_merged.columns[:10]].reset_index()

In [97]:
print(cluster1.shape)
cluster1.head(10)

(166, 11)


,index,Neighborhood,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport
0,1,Annadale,1,0,8,1,1,0,0,2,1
1,2,Arden Heights,0,0,2,0,2,0,0,1,1
2,3,Arlington,0,0,2,0,0,0,0,0,2
3,4,Arrochar,0,0,10,0,0,0,0,3,5
4,5,Arverne,0,0,4,0,2,0,0,1,3
5,7,Astoria Heights,1,0,6,0,2,0,0,2,1
6,8,Auburndale,0,0,7,2,1,0,0,7,1
7,13,Baychester,1,0,9,0,0,0,0,7,0
8,15,Bayswater,1,0,0,0,2,0,0,1,0
9,18,Beechhurst,0,0,8,0,0,0,0,3,0


In [96]:
print(cluster3.shape)
cluster3.head(10)

(80, 11)


,index,Neighborhood,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport
0,0,Allerton,1,0,16,0,1,0,0,11,2
1,9,Bath Beach,0,0,29,1,1,0,0,13,1
2,12,Bay Terrace,1,0,15,0,0,0,0,28,1
3,16,Bedford Park,1,0,23,2,1,0,0,10,4
4,17,Bedford Stuyvesant,0,0,14,4,1,0,0,6,0
5,23,Bensonhurst,1,0,15,1,0,0,0,6,1
6,31,Brighton Beach,0,0,21,1,5,1,0,13,0
7,38,Bulls Head,1,0,25,1,1,0,0,8,6
8,47,Central Harlem,7,0,23,2,1,2,0,5,0
9,52,City Line,0,0,19,0,0,0,0,23,1


In [95]:
print(cluster2.shape)
cluster2.head(10)

(54, 11)


,index,Neighborhood,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport
0,6,Astoria,2,0,62,16,0,0,0,13,0
1,10,Battery Park City,7,0,34,3,18,2,0,18,7
2,11,Bay Ridge,1,0,48,6,3,0,0,22,1
3,14,Bayside,1,0,42,10,1,0,0,14,1
4,22,Belmont,1,0,63,2,1,0,0,22,2
5,27,Boerum Hill,13,0,30,6,2,0,0,31,2
6,35,Brooklyn Heights,7,0,50,5,11,0,0,19,0
7,39,Bushwick,1,0,42,9,1,0,1,17,0
8,43,Carnegie Hill,7,0,46,9,1,1,0,24,0
9,44,Carroll Gardens,2,0,47,10,2,1,0,28,0
